# Emotion Detector

## Install Requirements and Mounting Google Drive

In [ ]:
#@title Mount Google Drive { vertical-output: true, display-mode: "form" }
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#@title Installing Libraries {vertical-output:true,display-mode:"form"}
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript, HTML
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization,AveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras import models
from tensorflow.keras.models import model_from_json
import h5py

cas=cv2.CascadeClassifier("/content/drive/MyDrive/Project Exhibition-II/1.XML")

print('Libraries Installed')

Libraries Installed


### Helper Function

In [ ]:
#@title BBox to Bytes {vertical-output:true,display-mode:"form"}
def bbox_to_bytes(bbox_array):
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
#@title WebCam Image Capture {vertical-output:true,display-mode:"form"}
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  img = js_to_image(data)
  return img

In [ ]:
#@title WebCam Video Stream {vertical-output:true,display-mode:"form"}
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
#@title Js to CV2 {vertical-output:true,display-mode:"form"}
def js_to_image(js_reply):
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

##V1.0

In [ ]:
#@title {vertical-output:true,display-mode:"form"}
#@markdown Use WebCam for Capture?
WEBCAM = True #@param {type:"boolean"}
if WEBCAM :
  img=take_photo()
else:
  #@markdown Enter the Address/Path of the Image File
  IMAGE_PATH="/content/drive/MyDrive/Project Exhibition-II/test_manual/IMG_20221019_175625 (2).jpg"#@param {type:"string"}
  img=cv2.imread(IMAGE_PATH)

gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
fac=cas.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=8)

for (x,y,w,h) in fac:
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

#@markdown Save Output to Google Drive (Default: false)?
SAVE = False #@param {type:"boolean"}

if(len(fac)>0):
  print("Face Detected")
  if(SAVE):
    cv2.imwrite('/content/drive/MyDrive/Project Exhibition-II/output/Face_Detected.jpg',img)
  cv2_imshow(img)
else:
  print("No Face Detected")

## v2.0

In [ ]:
#@title Trainer {vertical-output:true,display-mode:"form"}
img_features = []
img_label = []

print("Training Started")
def create_train_data():
    people = ['angry','disgust','fear', 'happy', 'neutral','sad','surprise']
    directory = r"/content/drive/MyDrive/Project Exhibition-II/train"

    for person in people:
        path_for_person=os.path.join(directory,person)
        img_labels=people.index(person)

        for img in os.listdir(path_for_person):
            img_path= os.path.join(path_for_person,img)

            img_array=cv2.imread(img_path)
            gray_image=cv2.cvtColor(img_array,cv2.COLOR_BGR2GRAY)

            faces_rect= cas.detectMultiScale(gray_image,scaleFactor=1.1,minNeighbors=2)

            for (x,y,w,h) in faces_rect:
                faces_roi=gray_image[y:y+h,x:x+w]
                img_features.append(faces_roi)
                img_label.append(img_labels)

create_train_data()

img_features=np.array(img_features,dtype='object')
img_label=np.array(img_label)

print("Size of Dataset: ", len(img_features))

face_recogniszer=cv2.face.LBPHFaceRecognizer_create()

face_recogniszer.train(img_features,img_label)

face_recogniszer.save('/content/drive/MyDrive/Project Exhibition-II/face_trained.yml')

np.save('/content/drive/MyDrive/Project Exhibition-II/features.npy',img_features)
np.save('/content/drive/MyDrive/Project Exhibition-II/labels.npy',img_label)

print("Trained")

In [ ]:
#@title Load Trained Model {vertical-output:true,display-mode:"form"}
features = np.load('/content/drive/MyDrive/Project Exhibition-II/features.npy',allow_pickle=True)
labels = np.load('/content/drive/MyDrive/Project Exhibition-II/labels.npy')

face_recogniszer=cv2.face.LBPHFaceRecognizer_create()
face_recogniszer.read('/content/drive/MyDrive/Project Exhibition-II/face_trained.yml')

people = ['angry','disgust','fear', 'happy', 'neutral','sad','surprise']

print('Model Loaded')

Model Loaded


In [ ]:
#@title Image { vertical-output: true, display-mode: "form" }
#@markdown Use WebCam for Capture?
WEBCAM = True #@param {type:"boolean"}
if WEBCAM:
  img=take_photo()
else:
  #@markdown Enter the Address/Path of the Image File
  IMAGE_PATH="/content/drive/MyDrive/Project Exhibition-II/test_manual/IMG_20221019_175625 (2).jpg"#@param {type:"string"}
  img=cv2.imread(IMAGE_PATH)

imggray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
fac=cas.detectMultiScale(imggray,scaleFactor=1.1,minNeighbors=7)

for (x,y,w,h) in fac:
    faces_roi=imggray[y:y+h,x:x+w]
    img_label,confidence=face_recogniszer.predict(faces_roi)
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),thickness=2)
    cv2.putText(img,str(people[img_label]),(x,y),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,255),thickness=1)

#@markdown Save Output to Google Drive (Default: false)?
SAVE = False #@param {type:"boolean"}

if(len(fac)>0):
  print("Face Detected")
  if(SAVE):
    cv2.imwrite('/content/drive/MyDrive/Project Exhibition-II/output/Face_Detected.jpg',img)
  cv2_imshow(img)
else:
  print("No Face/Emotion Detected")

In [ ]:
#@title Video { vertical-output: true, display-mode: "form" }
cas=cv2.CascadeClassifier("/content/drive/MyDrive/Project Exhibition-II/1.XML")

#@markdown Enter the Address/Path of the Video File
VIDEO_PATH="/content/drive/MyDrive/Project Exhibition-II/test_manual/WhatsApp Video 2023-01-18 at 15.44.37.mp4"#@param {type:"string"}
cap=cv2.VideoCapture(VIDEO_PATH)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_out = cv2.VideoWriter('/content/drive/MyDrive/Project Exhibition-II/output/Emotion_Detected_Vid.mp4', fourcc, 24, (w, h))
detected=False

success,img=cap.read()
while success:
    imggray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fac=cas.detectMultiScale(imggray,scaleFactor=1.1,minNeighbors=3)
    
    if(len(fac)>0): detected=True
    
    for (x,y,w,h) in fac:
        faces_roi=imggray[y:y+h,x:x+w]
        img_lable,confidence=face_recogniszer.predict(faces_roi)
        cv2.putText(img,str(people[img_lable]),(20,20),cv2.FONT_HERSHEY_COMPLEX,1.0,(0,255,0),thickness=2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),thickness=2)
    video_out.write(img)
    success,img=cap.read()
if(detected):
    video_out.release()
    print("Emotion Detected")
else:
    print("No Emotion Detected")

Emotion Detected


In [ ]:
#@title WebCam Feed { vertical-output: true , display-mode: "form" }
#@markdown Save WebCam Feed as video file (Default: false)?
SAVE = True #@param {type:"boolean"}
video_stream()
label_html='No Emotion Detected'
bbox = ''

if SAVE:
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  video = cv2.VideoWriter('/content/drive/MyDrive/Project Exhibition-II/output/Emotion_Detected_WebCam.mp4', fourcc, 24, (640, 480))

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    img = js_to_image(js_reply["img"])
    imggray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fac=cas.detectMultiScale(imggray,scaleFactor=1.1,minNeighbors=3)

    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    
    for (x,y,w,h) in fac:
      faces_roi=imggray[y:y+h,x:x+w]
      img_lable,confidence=face_recogniszer.predict(faces_roi)
      bbox_array = cv2.putText(bbox_array,str(people[img_lable]),(20,20),cv2.FONT_HERSHEY_COMPLEX,1.0,(0,255,0),thickness=2)
      bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)
      if SAVE:
        cv2.putText(img,str(people[img_lable]),(20,20),cv2.FONT_HERSHEY_COMPLEX,1.0,(0,255,0),thickness=2)
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),thickness=2)
    
    if SAVE:
      video.write(img)
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    
    bbox_bytes = bbox_to_bytes(bbox_array)
    
    bbox = bbox_bytes

    if(len(fac)>0): label_html = 'Emotion Detected'
    else: label_html='No Emotion Detected'

if SAVE:
  video.release()
  print("WebCam feed saved successfully")

## V3.0

In [ ]:
#@title Dataset Extraction {vertical-output:true, display-mode:"form"}
!unzip /content/archive_angry.zip -d /content/angry

In [ ]:
#@title Trainer {vertical-output:true, display-mode:"form"}
train_dir = "/content/train"
test_dir = "/content/test"

model = tf.keras.models.Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='tanh', input_shape=(48, 48, 1)))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (5, 5), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.75))

model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(6, activation='softmax'))

model.summary()

img_size=48

train_datagen = ImageDataGenerator(width_shift_range = 0.1,
                                         height_shift_range = 0.1,
                                         horizontal_flip = True,
                                         rescale = 1./255,
                                         validation_split = 0.2
                                        )
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                         validation_split = 0.2)
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (img_size,img_size),
                                                    batch_size = 32,
                                                    color_mode = "grayscale",
                                                    class_mode = "categorical",
                                                    subset = "training",
                                                    shuffle = True
                                                   )
validation_generator = validation_datagen.flow_from_directory( directory = test_dir,
                                                              target_size = (img_size,img_size),
                                                              batch_size = 32,
                                                              color_mode = "grayscale",
                                                              class_mode = "categorical",
                                                              subset = "validation",
                                                              shuffle = True
                                                             )
model.compile(
    optimizer=Adam(lr=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
  )
#@markdown Provide the name for the trained model.
MODEL_NAME="model_angry"#@param {type:"string"}
es=EarlyStopping(monitor="accuracy",min_delta=0.0001,patience=20,verbose=1,restore_best_weights=True)
mp=ModelCheckpoint(filepath=MODEL_NAME+".h5",monitor="accuracy",save_best_only=True,verbose=1)
es4=EarlyStopping(monitor="val_accuracy",min_delta=0.0001,patience=20,verbose=1,restore_best_weights=True)
mp4=ModelCheckpoint(filepath=MODEL_NAME+".h5",monitor="val_accuracy",save_best_only=True,verbose=1)
epochs = 200
batch_size = 32
history = model.fit(x=train_generator,epochs = epochs,validation_data = validation_generator,callbacks=[es,mp,es4,mp4])

model_save=model.to_json()
with open("/content/"+MODEL_NAME+".json","w") as json_file:
    json_file.write(model_save)

model.save_weights('/content/emotion_'+MODEL_NAME+'.h5')


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 conv2d_6 (Conv2D)           (None, 48, 48, 64)        18496     
                                                                 
 batch_normalization_6 (Batc  (None, 48, 48, 64)       256       
 hNormalization)                                                 
                                                                 
 average_pooling2d_4 (Averag  (None, 24, 24, 64)       0         
 ePooling2D)                                                     
                                                                 
 dropout_6 (Dropout)         (None, 24, 24, 64)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 24, 24, 128)      

FileNotFoundError: ignored

In [ ]:
#@title Load Trained Model {vertical-output: true, display-mode: "form"}
#@markdown Enter name of model to be used.
MODEL="model6"#@param {type:"string"}
json_file=open('/content/'+MODEL+'.json','r')
loded_file=json_file.read()
json_file.close()
emotion_model=model_from_json(loded_file)

emotion_model.load_weights('/content/emotion_'+MODEL+'.h5')
emotionsdictionary={0:"angry",1:"fear",2:"happy",3:"neutral",4:"sad",5:"surprise"}
print("Model Loaded")

Model Loaded


In [ ]:
#@title Image {vertical-output:true, display-mode: "form"}
#@markdown Use WebCam for Capture?
WEBCAM = False #@param {type:"boolean"}
if WEBCAM:
  img=take_photo()
else:
  #@markdown Enter the Address/Path of the Image File
  IMAGE_PATH="/content/download.jpeg"#@param {type:"string"}
  img=cv2.imread(IMAGE_PATH)

imggray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

fac=cas.detectMultiScale(imggray,scaleFactor=1.1,minNeighbors=7)


for (x,y,w,h) in fac:
    roi_gray_frame=imggray[y:y+h,x:x+w]
    cropped_img=np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame,(48,48)),-1),0)
    emotionpredict = emotion_model.predict(cropped_img)
    maxindex=int(np.argmax(emotionpredict))
    faces_roi=imggray[y:y+h,x:x+w]
    emotionpredict=emotion_model.predict(cropped_img)
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),thickness=2)
    cv2.putText(img,emotionsdictionary[maxindex],(x,y),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),thickness=2)

#@markdown Save Output to Google Drive (Default: false)?
SAVE = True #@param {type:"boolean"}

if(len(fac)>0):
  print("Face Detected")
  if(SAVE):
    cv2.imwrite('/content/drive/MyDrive/Project Exhibition-II/output/Face_Detected.jpg',img)
  cv2_imshow(img)
else:
  print("No Face/Emotion Detected")

In [ ]:
#@title Video { vertical-output: true, display-mode: "form" }
cas=cv2.CascadeClassifier("/content/drive/MyDrive/Project Exhibition-II/1.XML")

#@markdown Enter the Address/Path of the Video File
VIDEO_PATH="/content/drive/MyDrive/Project Exhibition-II/test_manual/WhatsApp Video 2023-01-18 at 15.44.37.mp4"#@param {type:"string"}
cap=cv2.VideoCapture(VIDEO_PATH)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter('/content/drive/MyDrive/Project Exhibition-II/output/Emotion_Detected_Tensorflow_Vid.mp4', fourcc, 24, (w, h))
detected=False

success,img=cap.read()
while success:
    imggray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fac=cas.detectMultiScale(imggray,scaleFactor=1.1,minNeighbors=3)
    
    if(len(fac)>0): detected=True
    
    for (x,y,w,h) in fac:
        roi_gray_frame=imggray[y:y+h,x:x+w]
        cropped_img=np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame,(48,48)),-1),0)
        emotionpredict = emotion_model.predict(cropped_img)
        maxindex=int(np.argmax(emotionpredict))
        faces_roi=imggray[y:y+h,x:x+w]
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),thickness=2)
        cv2.putText(img,emotionsdictionary[maxindex],(x,y),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),thickness=2)
    video.write(img)
    success,img=cap.read()
if(detected):
    print("Emotion Detected")
    video.release()
else:
    print("No Emotion Detected")

In [ ]:
#@title WebCam Feed { vertical-output: true , display-mode: "form" }
#@markdown Save WebCam Feed as video file (Default: false)?
SAVE = False #@param {type:"boolean"}
video_stream()
label_html='No Emotion Detected'
bbox = ''

if SAVE:
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  video = cv2.VideoWriter('/content/drive/MyDrive/Project Exhibition-II/output/Emotion_Detected_Tensorflow_WebCam.mp4', fourcc, 24, (640, 480))

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    img = js_to_image(js_reply["img"])
    imggray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fac=cas.detectMultiScale(imggray,scaleFactor=1.1,minNeighbors=9)

    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    
    for (x,y,w,h) in fac:
      roi_gray_frame=imggray[y:y+h,x:x+w]
      cropped_img=np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame,(48,48)),-1),0)
      emotionpredict = emotion_model.predict(cropped_img,verbose=0)
      maxindex=int(np.argmax(emotionpredict))
      faces_roi=imggray[y:y+h,x:x+w]
      bbox_array = cv2.putText(bbox_array,emotionsdictionary[maxindex],(x,y),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),thickness=2)
      bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(0,255,0),thickness=2)
      if SAVE:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),thickness=2)
        cv2.putText(img,emotionsdictionary[maxindex],(x,y),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),thickness=2)
    
    if SAVE:
      video.write(img)
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    
    bbox_bytes = bbox_to_bytes(bbox_array)
    
    bbox = bbox_bytes

    if(len(fac)>0): label_html = 'Emotion Detected'
    else: label_html='No Emotion Detected'

if SAVE:
  video.release()
  print("WebCam feed saved successfully")

<IPython.core.display.Javascript object>